In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
import os
import pandas as pd
from scipy.signal import stft
from math import prod
# from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from math import e
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from urllib.request import urlopen, urlretrieve
from itertools import combinations

# import matplotlib.pyplot as plt
%matplotlib inline

2022-06-16 16:21:52.800926: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-16 16:21:52.804459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-16 16:21:52.804470: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def butter_bandpass(data, lowcut, highcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_lowpass(data, lowcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)


def butter_highpass(data, highcut, fs=200, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=200, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

In [3]:
def filtros(data):
    data_filtered = butter_notch(data, 60)
    data_filtered = butter_highpass(data_filtered, 5)
    data_filtered = butter_lowpass(data_filtered, 50)
    return data_filtered

In [4]:
dir = './datasets/topicos_cc'
arquivos = os.listdir(dir)
arq_numpy = [f for f in arquivos if f.endswith(".npy") and f.startswith('p2')]
participantes = {}
vetor = []
for i in arq_numpy:
    nome = i.split('_')
    trial = np.load(dir+'/'+i)
    for m in range(0,8):
        if participantes.get(f'trial_{nome[1]}',0) == 0:
            participantes[f'trial_{nome[1]}'] = []
        dados = trial[m, :, :].swapaxes(0,1)
        participantes[f'trial_{nome[1]}'].append(filtros(dados))
        

arr = np.vstack((np.array(participantes['trial_1']), np.array(participantes['trial_2']), np.array(participantes['trial_3'])))


print(arr.shape)
participantes

(24, 4, 1600)


{'trial_2': [array([[  0.29706151,  -0.10758804,   1.23396836, ..., -38.45522264,
          -17.73104717,   0.81996573],
         [  0.08937505,  -7.59702407,  -5.33727604, ...,  30.65703983,
           39.85397463,  -4.36759111],
         [ -0.37811694,   0.44031343,   3.97210754, ...,  14.22481131,
           26.12198471,   2.06869271],
         [  0.08187095,   2.26459197,   3.72874838, ...,  -5.42794101,
           -2.58438479,  -0.51082867]]),
  array([[-1.73949608e-01, -1.42606709e+01, -1.55178994e+01, ...,
          -5.04905317e+01, -4.19364713e+01,  9.20553298e+00],
         [ 3.61063262e-02,  3.88964943e+00,  2.20453159e+00, ...,
           7.98866294e+01,  7.50537834e+01, -2.10478914e-01],
         [ 4.99892252e-02, -5.66187924e+00, -9.97914406e+00, ...,
           2.21823731e+00,  8.47436948e-01,  3.28411724e-01],
         [ 2.29972170e-02,  3.26872918e+00,  4.23690585e+00, ...,
          -6.01274568e+00, -2.61444681e+00, -1.98066580e-01]]),
  array([[  0.11907216,   3.39327

In [5]:
step = 11.8
segment = 128
data = arr
# .get_data()
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * int(step)

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=128, noverlap=115)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes, ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes, ensaios, canais, janelas, linhas)')

 (24, 4, 1600)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (24, 4, 125, 128) - (classes, ensaios, canais, janelas, linhas)
Dominio da frequência:  (24, 4, 125, 65) - (classes, ensaios, canais, janelas, linhas)


In [6]:
# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape[:-1])))

def wamp(x):
    limiar = np.abs(np.diff(x))
    return np.sum(limiar > 0.0001, axis=-1)

def wl(x):
    return np.sum(np.abs(np.diff(x)), axis=-1)


def getzc(data, th):
    t = len(data)
    soma = 0
    for i in range(t-1):
        res = (data[i]*data[i+1])
        res2 = np.abs(data[i]-data[i+1])
        if (res<0 and res2 > th):
            soma +=1
    return soma

def zc(data):
    f=[]
    x,y,z = data.shape[:3]
    for i in range(x):
        l = []
        for j in range(y):
            li = []
            for k in range(z):
                li.append(getzc(data[i][j][k], 0.0001))
            l.append(li.copy())
        f.append(l.copy())

    return np.array(f)

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2


def fmn(w):
    sample_rate = 200
    f = (w * sample_rate)/(2*len(w))
    return np.sum(np.abs(f*PSD(w)), axis=-1)/np.sum(PSD(w), axis=-1)

def mmnf(w):
    sample_rate = 200
    f = (w * sample_rate)/(2*len(w))
    return np.sum(np.abs(f*np.abs(w)), axis=-1)/np.sum(np.abs(w), axis=-1)


def logDec(data):
    N = np.prod(data.shape)
    return e ** (np.sum(np.log10(np.abs(data)), axis=-1))/N


def iemg(x):
    return np.sum(np.abs(x), axis=-1)

def dasdv(x):
    return np.sqrt(np.sum(np.diff(x)**2, axis=-1)/(np.prod(x.shape[:-1]) - 1))

def tm(x,n):
    return np.abs(np.sum(x**n , axis=-1)/np.prod(x.shape[:-1]))

In [7]:
final_data = list()
final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))
final_data.append(fmd(chunks_freq))
final_data.append(mmdf(chunks_freq))
final_data.append(logDec(chunks_time))
final_data.append(wamp(chunks_time))
final_data.append(wl(chunks_time))
final_data.append(zc(chunks_time))
final_data.append(fmn(chunks_freq))
final_data.append(mmnf(chunks_freq))
final_data.append(iemg(chunks_time))
final_data.append(dasdv(chunks_time))

for n in range(3,6):
    final_data.append(tm(chunks_time, n))

f, Pxx_den = signal.welch(data, fs=200, nperseg=248, noverlap=223)
final_data.append(Pxx_den)

final = np.array(final_data)
final.shape


(16, 24, 4, 125)

In [8]:
X = final.transpose(1,3,2,0)
X = X.reshape(X.shape[0]*X.shape[1], X.shape[2]*X.shape[3])

In [9]:
y = [[i] * int(X.shape[0] / 8) for i in range(8)]
y = np.array(y).flatten()

In [10]:
trial = []
trial.append((X[:1000], y[:1000]))
trial.append((X[1000:2000], y[1000:2000]))
trial.append((X[2000:3000], y[2000:3000]))

In [11]:
def plot_history(h):
    loss_list = [s for s in h.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in h.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in h.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in h.history.keys() if 'acc' in s and 'val' in s]
    if len(loss_list) == 0:
        print('Custo não está presente no histórico')
        return
    epochs = range(1, len(history.history[loss_list[0]]) + 1)
    # Custo
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, h.history[l], 'b',
                 label='Custo [treinamento] (' + str(str(format(
                    h.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, h.history[l], 'g',
                 label='Custo [validação] (' + str(str(format(
                    h.history[l][-1],'.5f'))+')'))
    plt.title('Custo')
    plt.xlabel('Épocas')
    plt.ylabel('Custo')
    plt.legend()
    # Acurácia
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, h.history[l], 'b',
                 label='Acurácia [treinamento] (' + str(format(
                    h.history[l][-1],'.5f'))+')')
    for l in val_acc_list:
        plt.plot(epochs, h.history[l], 'g',
                 label='Acurácia [validação] (' + str(format(
                    h.history[l][-1],'.5f'))+')')
    plt.title('Acurácia')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.show()

In [12]:
def cross_val(model):
    j=0
    scores = []
    validations=[]
    for i in combinations(trial, 2):
        X_treino = np.vstack((i[0][0],i[1][0]))
        X_teste = trial[j][0]

        y_treino = np.vstack((i[0][1],i[1][1]))
        y_treino= y_treino.reshape(y_treino.shape[0]*y_treino.shape[1])
        y_teste = trial[j][1]


        j+=1

        X_treino, X_val, y_treino, y_val = train_test_split(
        X_treino, y_treino, test_size=0.3, shuffle=True, stratify=y_treino)

        scaler = StandardScaler()
        scaler.fit(X_treino)

        X_treino = scaler.transform(X_treino)
        X_teste = scaler.transform(X_teste)
        X_val = scaler.transform(X_val)

        history = model.fit(X_treino, y_treino, epochs=100, batch_size=15, validation_data=(X_val, y_val))
        # plot_history(history)
        predict_x=model.predict(X_val) 
        score=np.argmax(predict_x,axis=1)
        scores.append(score)
        validations.append(y_val)


    # y_true =  np.vstack((trial[0][1],trial[1][1], trial[2][1]))
    y_true =  np.vstack((validations[0],validations[1], validations[2]))
    y_true= y_true.reshape(y_true.shape[0]*y_true.shape[1])
    sc = np.vstack((scores[0], scores[1], scores[2]))
    sc= sc.reshape(sc.shape[0]*sc.shape[1])

    print('\n\nAcurácia: %0.2f%%' % (accuracy_score(y_true, sc) * 100))
    print('Matriz de confusão:')
    print(confusion_matrix(y_true, sc))
    print()
    print(classification_report(y_true, sc, digits=5))

In [13]:
# definição de uma fração do regularizador
l = 0.01

# desenvolvimento do modelo Keras para uma MLP
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=64,
                kernel_regularizer=regularizers.l2(l)))
# Aplicação de um dropout (caso necessário)
# model.add(Dropout(0.5))
model.add(Dense(10, activation='relu',
                kernel_regularizer=regularizers.l2(l)))
# Aplicação de um dropout (caso necessário)
# model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

# Aplicação de um modelo de descida de gradiente utilizando o Stocastic Gradient Descendent (SGD)
sgd = SGD(lr=0.05, momentum=0.0)
# Função de otimização da rede: ADAM
adam = Adam(lr=0.005, beta_1=0.9, beta_2=0.999)
# Função de custo baseada em dados originalmente categóricos


model.compile(loss='sparse_categorical_crossentropy', optimizer=adam,
              metrics=['accuracy'])


  
cross_val(model)

Epoch 1/100


2022-06-16 16:21:56.313335: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-16 16:21:56.313359: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-16 16:21:56.313377: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (luisotavio-Aspire5): /proc/driver/nvidia/version does not exist
2022-06-16 16:21:56.313653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/luisotavio/.local/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gr

94/94 [==============================] - 1s 2ms/step - loss: 2.0064 - accuracy: 0.3150 - val_loss: 1.6263 - val_accuracy: 0.4333
Epoch 2/100
94/94 [==============================] - 0s 1ms/step - loss: 1.4984 - accuracy: 0.4786 - val_loss: 1.3589 - val_accuracy: 0.4933
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 1.3173 - accuracy: 0.5329 - val_loss: 1.2479 - val_accuracy: 0.5767
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 1.1993 - accuracy: 0.6186 - val_loss: 1.1419 - val_accuracy: 0.6533
Epoch 5/100
94/94 [==============================] - 0s 1ms/step - loss: 1.1518 - accuracy: 0.6500 - val_loss: 1.1396 - val_accuracy: 0.6233
Epoch 6/100
94/94 [==============================] - 0s 2ms/step - loss: 1.0862 - accuracy: 0.6950 - val_loss: 1.0958 - val_accuracy: 0.6250
Epoch 7/100
94/94 [==============================] - 0s 1ms/step - loss: 1.0668 - accuracy: 0.6986 - val_loss: 1.0199 - val_accuracy: 0.7083
Epoch 8/100
94/94 [======

In [14]:
def cross_val2(model):
    j=0
    scores = []
    validations = []
    for i in combinations(trial, 2):
        X_treino = np.vstack((i[0][0],i[1][0]))
        X_teste = trial[j][0]

        y_treino = np.vstack((i[0][1],i[1][1]))
        y_treino= y_treino.reshape(y_treino.shape[0]*y_treino.shape[1])
        y_teste = trial[j][1]


        j+=1

        X_treino, X_val, y_treino, y_val = train_test_split(
        X_treino, y_treino, test_size=0.3, shuffle=True, stratify=y_treino)

        scaler = StandardScaler()
        scaler.fit(X_treino)

        X_treino = scaler.transform(X_treino)
        X_teste = scaler.transform(X_teste)
        X_val = scaler.transform(X_val)

        history = model.fit(X_treino, y_treino)
        # plot_history(history)
        predict_x=model.predict(X_val) 
        # print(predict_x)
        # print(predict_x.shape)
        # score=np.argmax(predict_x)
        # score=np.argmax(predict_x,axis=1)
        # scores.append(score)
        scores.append(predict_x)
        validations.append(y_val)

    y_true =  np.vstack((validations[0],validations[1], validations[2]))
    y_true= y_true.reshape(y_true.shape[0]*y_true.shape[1])
    sc = np.vstack((scores[0], scores[1], scores[2]))
    sc= sc.reshape(sc.shape[0]*sc.shape[1])

    print('\n\nAcurácia: %0.2f%%' % (accuracy_score(y_true, sc) * 100))
    print('Matriz de confusão:')
    print(confusion_matrix(y_true, sc))
    print()
    print(classification_report(y_true, sc, digits=5))

In [15]:

# modelo de classificador com os parâmetros padrões
clf = SVC(gamma='scale')

# criando o modelo de classificação com os dados de treino
cross_val2(clf)



Acurácia: 67.33%
Matriz de confusão:
[[169  17  19   8   4   2   4   2]
 [  1 120  13  28  15   0  37  12]
 [  8  18 151   4  20  13   0  11]
 [  8   4   8 157   5   9  33   1]
 [  5  19   5  26 126   2  28  14]
 [ 12   3   8  12   2 138  36  13]
 [  0   1   0   4   0   2 213   5]
 [ 10  11  10  10  17  13  16 138]]

              precision    recall  f1-score   support

           0    0.79343   0.75111   0.77169       225
           1    0.62176   0.53097   0.57279       226
           2    0.70561   0.67111   0.68793       225
           3    0.63052   0.69778   0.66245       225
           4    0.66667   0.56000   0.60870       225
           5    0.77095   0.61607   0.68486       224
           6    0.58038   0.94667   0.71959       225
           7    0.70408   0.61333   0.65558       225

    accuracy                        0.67333      1800
   macro avg    0.68417   0.67338   0.67045      1800
weighted avg    0.68409   0.67333   0.67039      1800

